# Lesson 9 - Creating an Agentic multi-agent system using A2A with BeeAI Framework

In this final code lesson, you will create a comprehensive "Healthcare Concierge" system. You will use the **BeeAI Framework** to orchestrate all three agents you have built so far (Policy, Research, and Provider). The BeeAI `RequirementAgent` will act as a router, deciding which A2A agent to hand off to based on the user's complex query.


In [ ]:
import asyncio
import os

from IPython.display import Markdown, display
from dotenv import load_dotenv

from helpers import authenticate

## 9.1. Start All Agent Servers

Ensure all three terminals are running their respective agents:
1.  **Policy Agent** (Lesson 3)
2.  **Research Agent** (Lesson 5)
3.  **Provider Agent** (Lesson 7)


In [ ]:
from IPython.display import IFrame

url = os.environ.get("DLAI_LOCAL_URL").format(port=8888)
IFrame(f"{url}terminals/1", width=800, height=200)

In [ ]:
IFrame(f"{url}terminals/2", width=800, height=200)

In [ ]:
IFrame(f"{url}terminals/3", width=800, height=200)

## 9.2. Define BeeAI Components

Here you will:
1.  Import BeeAI framework components, including `RequirementAgent` and `HandoffTool`.
2.  Define `A2AAgent` instances for each of your running servers.
3.  Use `check_agent_exists()` to fetch the metadata (AgentCard) from each server.


In [ ]:
from beeai_framework.adapters.a2a import A2AServer, A2AServerConfig
from beeai_framework.adapters.a2a.agents import A2AAgent
from beeai_framework.adapters.vertexai import VertexAIChatModel
from beeai_framework.agents.requirement import RequirementAgent
from beeai_framework.agents.requirement.requirements.conditional import (
    ConditionalRequirement,
)
from beeai_framework.memory import UnconstrainedMemory
from beeai_framework.memory.unconstrained_memory import UnconstrainedMemory
from beeai_framework.serve.utils import LRUMemoryManager
from beeai_framework.tools.handoff import HandoffTool
from beeai_framework.tools.think import ThinkTool

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
load_dotenv()
_, project_id = authenticate()

In [ ]:
host = os.environ.get("AGENT_HOST")
policy_port = os.environ.get("POLICY_AGENT_PORT")
research_port = os.environ.get("RESEARCH_AGENT_PORT")
provider_port = os.environ.get("PROVIDER_AGENT_PORT")
healthcare_agent_port = int(os.environ.get("HEALTHCARE_AGENT_PORT"))

In [ ]:
policy_agent = A2AAgent(
    url=f"http://{host}:{policy_port}", memory=UnconstrainedMemory()
)
# Run `check_agent_exists()` to fetch and populate AgentCard
asyncio.run(policy_agent.check_agent_exists())
print("\tℹ️", f"{policy_agent.name} initialized")

In [ ]:
research_agent = A2AAgent(
    url=f"http://{host}:{research_port}", memory=UnconstrainedMemory()
)
asyncio.run(research_agent.check_agent_exists())
print("\tℹ️", f"{research_agent.name} initialized")

In [ ]:
provider_agent = A2AAgent(
    url=f"http://{host}:{provider_port}", memory=UnconstrainedMemory()
)
asyncio.run(provider_agent.check_agent_exists())
print("\tℹ️", f"{provider_agent.name} initialized")

## 9.3. Configure the Orchestrator (Healthcare Concierge)

You will now configure the `RequirementAgent`. This agent uses a `VertexAIChatModel` and is equipped with `HandoffTool`s connected to your A2A agents. The instructions explicitly guide the LLM on how to use each specific agent (Research for conditions, Policy for insurance, Provider for doctors) to answer multi-part questions.


In [ ]:
healthcare_agent = RequirementAgent(
    name="Healthcare Agent",
    description="A personal concierge for Healthcare Information, customized to your policy.",
    llm=VertexAIChatModel(
        model_id="gemini-2.5-flash",
        project=project_id,
        location="global",
        allow_parallel_tool_calls=True,
    ),
    tools=[
        ThinkTool(),
        HandoffTool(
            policy_agent,
            name=policy_agent.name,
            description=policy_agent.agent_card.description,
        ),
        HandoffTool(
            research_agent,
            name=research_agent.name,
            description=research_agent.agent_card.description,
        ),
        HandoffTool(
            provider_agent,
            name=provider_agent.name,
            description=provider_agent.agent_card.description,
        ),
    ],
    requirements=[
        ConditionalRequirement(ThinkTool, consecutive_allowed=False),
    ],
    role="Healthcare Concierge",
    instructions=(
        f"""You are a concierge for healthcare services. Your task is to handoff to one or more agents to answer questions and provide a detailed summary of their answers. Be sure that all of their questions are answered before responding.
        Use `{research_agent.name}` for research about their condition. Provide this research to the user.
        Use `{policy_agent.name}` for information about their specific Health Insurance Policy.
        Use `{provider_agent.name}` for finding providers in their location.

        IMPORTANT: When returning answers about providers, only output providers from `{provider_agent.name}` and only provide insurance information based on the results from `{policy_agent.name}`.

        In your output, put which agent gave you the information."""
    ),
)

print("\tℹ️", f"{healthcare_agent.meta.name} initialized")

## 9.4. Run the Full Workflow

Test the system with a complex query that requires information from all three sub-agents.


In [ ]:
response = await healthcare_agent.run(
    "I'm based in Austin, TX. How do I get mental health therapy near me and what does my insurance cover?"
)
display(Markdown(response.last_message.text))

## 9.5. Serve the Concierge Agent

Finally, you can register this high-level "Concierge" agent itself as an A2A server. This demonstrates the recursive power of A2A: an agent composed of other A2A agents can itself be exposed as an A2A agent.


In [ ]:
# Register the agent with the A2A server and run the HTTP server
# we use LRU memory manager to keep limited amount of sessions in the memory
server = A2AServer(
    config=A2AServerConfig(host=host, port=healthcare_agent_port),
    memory_manager=LRUMemoryManager(maxsize=100),
).register(healthcare_agent)

server.serve()

## 9.6. Resources

- [BeeAI Framework](https://framework.beeai.dev/introduction/welcome)
- [Requirement Agent](https://framework.beeai.dev/modules/agents/requirement-agent)
- [BeeAI Framework GitHub](https://github.com/i-am-bee/beeai-framework)

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>
</div>
